# Prueba práctica 2 : Clasificación con datos del desastre del titanic


Utilizando lo aprendido en clases, se solicita lograr reponder la pregunta: `¿Podemos predecir que persona sobrevivió en el titanic?`, para ello se entrega un set de datos con 891 registros a analizar donde la columna `Survived` indica si sobrevivió (valor 1) o si no lo hizo (valor 0).

Primero se debe leer bien la información entregada en el documento word que describe los distintos datos y lo que se solicita para la evaluación de la prueba


In [ ]:
import findspark
findspark.init()

import pyspark
sc = pyspark.SparkContext(appName="titanic")

from pyspark.sql import SQLContext, SparkSession 
sq = SQLContext(sc)
spark = SparkSession(sc)

findspark.find()

Cargamos el archivo proporcionado: `la locación depende de donde se tenga en el equipo`

In [ ]:
df = spark.read.csv('file:///C:/train.csv', header=True, inferSchema = True);
df.printSchema()

In [ ]:
import pandas as pd
datos = df.toPandas()
datos.loc[:,'Total'] = 0
print(datos.groupby(['Survived'],as_index=False).agg({"Total":"count"}))


Se realiza un análisis de `correlación de variables` para ver la relación que existe entre todas las variables numéricas y principalmente con la variable de supervivencia (eliminamos la variable PassengerId del estudio, dado que no aporta información producto que es un secuancial). Para ello debemos explorar la data graficando como se muestra con el siguiente código.

`En esta parte no debe agregar código sino solamente ejecutar`

In [ ]:
from pandas.plotting import scatter_matrix
print(df.dtypes)

drop_list = ['PassengerId']

df_c = df.select([column for column in df.columns if column not in drop_list])

numeric_features = [t[0] for t in df_c.dtypes if t[1] in ('int','double')]
df.select(numeric_features).describe().toPandas().transpose()



In [ ]:
numeric_data = df.select(numeric_features).toPandas()
axs = scatter_matrix(numeric_data, figsize=(8, 8));
n = len(numeric_data.columns)
for i in range(n):
    v = axs[i, 0]
    v.yaxis.label.set_rotation(0)
    v.yaxis.label.set_ha('right')
    v.set_yticks(())
    h = axs[n-1, i]
    h.xaxis.label.set_rotation(90)
    h.set_xticks(())

In [ ]:
import seaborn as sns; sns.set()
import matplotlib.pyplot as plt
corr = df.select(numeric_features).toPandas().corr()

sns.set(rc={"font.style":"normal",
            "axes.titlesize":30,
           "text.color":"black",
            "xtick.color":"black",
            "ytick.color":"black",
            "axes.labelcolor":"black",
            "axes.grid":False,
            'axes.labelsize':30,
            'figure.figsize':(8.0, 8.0),
            'xtick.labelsize':20,
            'ytick.labelsize':20})
sns.heatmap(corr,annot = True, annot_kws={"size": 15},cmap="viridis")

plt.show()

Ahora debemos revisar si en el set de datos vienen datos nulos para las variables, lo que no lleva a:
1.- Eliminar variables del estudio si la cantidad de nulos es muy alta que no se pueda determinar un valor de reemplazo
2.- Tratar los valores nulos de las variables, reemplazando estos valores por el valor mas ad-hoc considerando los demás datos de la variable

`En esta parte no debe agregar código sino solamente ejecutar`

In [ ]:
import numpy as np
missing_df = df.select([c for c in df]).toPandas().isnull().sum(axis=0).reset_index()
missing_df.columns = ['column_name', 'missing_count']
missing_df = missing_df.loc[missing_df['missing_count']>0]
missing_df = missing_df.sort_values(by='missing_count')
ind = np.arange(missing_df.shape[0])
width = 0.9
fig, ax = plt.subplots(figsize=(12,6))
rects = ax.barh(ind, missing_df.missing_count.values, color='blue')
ax.set_yticks(ind)
ax.set_yticklabels(missing_df.column_name.values, rotation='horizontal')
ax.set_xlabel("Cantidad de valores nulos")
ax.set_title("Número de valores nulos por cada variable")
plt.show()

De acuerdo a lo visto en los análisis anteriores podemos decidir que variables son necesarias para el estudio:
`En este punto ustedes pueden determinar que variables van a utilizar, les entregaré detalles de la revisión para que puedan decidir en base a una decisión particular o la que he tomado`

1.- La variable `PassengerId` es una secuencia que no nos aporta información para el estudio, sólo nos ayuda al final para identificar al pasajero que sobrevivió.
2.- La variable `Cabin`(aunque parece muy interesante), yo la descarto dado que tiene 687 valores vacíos de 891 posibles. (Si estan interesados en completarla hay que leer mas del titanic para ver como se dividen las cabinas por clases, considerando posibles soluciones de acuerdo a estos datos).
3.- La variable `Fare`, yo la descarto debido a que tiene fuerza en la correlación inversa con la variable `Pclass` (-0,55), pero la variable `Pclass` tiene mayor correlación inversa con la variable `Survived (que es la relación que nos interesa)` (-0,34). Por ello prefiero `Pclass` a `fare`.
`El punto 3 es de acuerdo a cada visión, hay entrenamientos que consideran las dos variables, hay otros que sólo fare`

`En este punto cada grupo puede decidir las variables a estudiar`.

A continuación un estudios para las variables para completar los valores nulos de las variables `Age` y `Embarked` que las encuentro interesantes para el estudio.

`En esta parte pueden agregar código para estudiar la o las variables que decidan utilizar o seguir el codigo proporcionado`

In [ ]:
media = df.select('Age').toPandas().mean().round(1)
print(media)
print(datos.groupby(['Embarked','Pclass'],as_index=False).agg({"Total":"count"}))
print(df.where(df.Embarked.isNull()).show())
print(df.toPandas().isnull().sum(axis=0))


En el detalle de lo anterior vemos lo siguiente:
1.- La variable `Age` tiene un promedio de 29,7 años, pero ello incluye todas las clases, puede ser una medida, pero en mi caso consideré el promedio de edad por clases (en la siguiente ejecución se reemplaza los nulos por el promedio de edad por clase). En un estudio mas avanzado podemos considerar mas variables para obtener una medida mas real para completar los valores nulos de edad.

2.- La variable `Embarked`, sólo tiene dos registros con valores nulos y ambos pertenecen a primera clase, por lo que tomaremos el valor mas repetido de la variable para la primera clase que en este caso es `S`


`En este punto ustedes pueden determinar otra manera de reemplazar los nulos o seguir el codigo proporcionado`

In [ ]:
from pyspark.sql.functions import col, when
prom_1 = df.select('Age').where(df.Pclass == 1).toPandas().mean().round(1)
prom_2 = df.select('Age').where(df.Pclass == 2).toPandas().mean().round(1)
prom_3 = df.select('Age').where(df.Pclass == 3).toPandas().mean().round(1)

df_temp = df.withColumn("Age",
        when((col("Pclass") == 1) & (col("Age").isNull()), float(prom_1))
        .when((col("Pclass") == 2) & (col("Age").isNull()), float(prom_2))
        .when((col("Pclass") == 3) & (col("Age").isNull()), float(prom_3))
        .otherwise(col("Age")))
df2 = df_temp.na.fill({'Embarked': 'S'})



In [ ]:
import numpy as np
missing_df = df2.select([c for c in df2]).toPandas().isnull().sum(axis=0).reset_index()
missing_df.columns = ['column_name', 'missing_count']
missing_df = missing_df.loc[missing_df['missing_count']>0]
missing_df = missing_df.sort_values(by='missing_count')
ind = np.arange(missing_df.shape[0])
width = 0.9
fig, ax = plt.subplots(figsize=(12,6))
rects = ax.barh(ind, missing_df.missing_count.values, color='blue')
ax.set_yticks(ind)
ax.set_yticklabels(missing_df.column_name.values, rotation='horizontal')
ax.set_xlabel("Cantidad de valores nulos")
ax.set_title("Número de valores nulos por cada variable")
plt.show()

Después de todo el análisis y reemplazo de valores nulos, comenzamos con la preparación de la data:

1.- Eliminando las columnas que no serán parte del estudio como: `PassengerId, Name, Ticket, Fare, Cabin`
2.- Se debe realizar el manejo de las variables string o categóricas para dejarlas en variables numéricas


`En esta parte debe proporcionar el código para realizar el entrenamiento del clasificador`

In [ ]:
# Se eliminan las columnas que no se incluiran en el estudio, usted puede dejar de eliminar columnas de acuerdo a percepción de 
# las variables importantes para el estudio

df2.printSchema()

drop_list = ['PassengerId', 'Name','Ticket','Fare','Cabin']

df_evaluar = df2.select([column for column in df.columns if column not in drop_list])
df_evaluar.printSchema()


In [ ]:
####Proporcionar el código para realizar el tratamiento de variables string a variable numérica, utilizar componentes que se 
#### muestran abajo.
from pyspark.ml.feature import OneHotEncoderEstimator, StringIndexer, VectorAssembler
data_string = ['Sex','Embarked']


stages = []
for dataCategorica in data_string:
    stringIndexer = StringIndexer(inputCol = dataCategorica, outputCol = dataCategorica  + 'Index')
    encoder = OneHotEncoderEstimator(inputCols=[stringIndexer.getOutputCol()], outputCols=[dataCategorica + "classVec"])
    stages += [stringIndexer, encoder]
    print(encoder.getOutputCols())

data_numerica = ['Pclass','Age','SibSp','Parch']
assemblerInputs = [c + "classVec" for c in data_string] + data_numerica
vector_features = VectorAssembler(inputCols=assemblerInputs, outputCol="features")
stages += [vector_features]
print(assemblerInputs)


In [ ]:
### Utilizar Pipeline para la transformación de la data y recordar renombrar la columna Survived a lavel

from pyspark.ml import Pipeline
pipeline = Pipeline(stages = stages)
pipelineModel = pipeline.fit(df_evaluar)
train_V = pipelineModel.transform(df_evaluar)
train_V = (train_V
       .withColumnRenamed('Survived','label'))
train_V.printSchema()
train_V.take(2)


In [ ]:
### Recordar dividir el set en entranamiento y testing e imprimir las cantidades de cada uno para corroborar la división

trainingData, testData = train_V.randomSplit([0.8, 0.2], seed=100)
print(trainingData.count())
print(testData.count())

In [ ]:
### Entrenar el clasificador Dde árbol de decisión, recuerde elegir a juicio experto la altura 

from pyspark.ml.classification import DecisionTreeClassifier

# Create initial Decision Tree Model
dt = DecisionTreeClassifier(labelCol="label", featuresCol="features", maxDepth=5)
dtModel = dt.fit(trainingData)
print("numNodes = ", dtModel.numNodes)
print("depth = ", dtModel.depth)
print(dtModel.toDebugString)

In [ ]:
### Realizar la predicción y seleccionar 10 para validar como se ve nuestra predicción

predictions = dtModel.transform(testData)
predictions.select('Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Embarked','label', 'prediction', 'probability').show(10)

In [ ]:
### Evaluar el clasificador, la medida que utilizaremos es la Accuracy, e imprimir la valoración

from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator = MulticlassClassificationEvaluator(
    labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test Error = %g" % (1.0 - accuracy))
print("Precisiòn = %g" % (accuracy*100))
evaluator.evaluate(predictions)

In [ ]:
## implementar validación cruzada (cross-validation) para ver si logramos mejora de la predicción.

from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
paramGrid = (ParamGridBuilder()
             .addGrid(dt.maxDepth, [3, 4, 5, 6, 7, 8, 9, 10])
            .addGrid(dt.maxBins, [10, 20, 30, 40, 50, 60, 70, 100])
             .build())

evaluatorCV = MulticlassClassificationEvaluator(
    labelCol="label", predictionCol="prediction", metricName="accuracy")
cv = CrossValidator(estimator=dt, estimatorParamMaps=paramGrid, evaluator=evaluatorCV, numFolds=7)
cvModel = cv.fit(trainingData)
print("numNodes = ", cvModel.bestModel.numNodes)
print("depth = ", cvModel.bestModel.depth)
print("depth = ", cvModel.bestModel.maxBins)

In [ ]:
### Realizar las predicciones con el modelo de validación cruzada (cross-validation) y calcula el accuracy del mejor modelo

predictionsCV = cvModel.transform(testData)
evaluatorcv = MulticlassClassificationEvaluator(
    labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracycv = evaluatorcv.evaluate(predictionsCV)
print("Test Error = %g" % (1.0 - accuracycv))
print("Precisiòn = %g" % (accuracycv*100))
evaluatorcv.evaluate(predictionsCV)

In [ ]:
### Implementar gráfica de la matriz de confusión de manera de validar cuantas prediciones fueron correctas.
### Seguir instrucciones del ejemplo realizado en clases

import matplotlib.pyplot as plt
import numpy as np
import itertools

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
class_temp = predictionsCV.select("label").groupBy("label")\
                        .count().sort('count', ascending=False).toPandas()
class_temp = class_temp["label"].values.tolist()
class_names = map(int, class_temp)
# # # print(class_name)
class_names

In [ ]:
from sklearn.metrics import confusion_matrix
y_true = predictionsCV.select("label")
y_true = y_true.toPandas()

y_pred = predictionsCV.select("prediction")
y_pred = y_pred.toPandas()

cnf_matrix = confusion_matrix(y_true, y_pred)

In [ ]:
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=list(class_names),
                      title='Matriz de confusión')
plt.show()

In [ ]:
### Implementar dos clasificadores adicionales para poder lograr una mejor accuracy. Puede elegir alguno de los vistos en clase.

from pyspark.ml.classification import RandomForestClassifier
rf = RandomForestClassifier(featuresCol = 'features', labelCol = 'label', numTrees=700)
rfModel = rf.fit(trainingData)
predictionsRF = rfModel.transform(testData)

In [ ]:
evaluatorRF = MulticlassClassificationEvaluator(
    labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracyRF = evaluatorRF.evaluate(predictionsRF)
print("Test Error = %g" % (1.0 - accuracyRF))
print("Precisiòn = %g" % (accuracyRF*100))
evaluatorRF.evaluate(predictionsRF)

In [ ]:
from pyspark.ml.classification import GBTClassifier, OneVsRest
gbt = GBTClassifier(maxIter=50)
gbtModel = gbt.fit(trainingData)
predictionsgbt = gbtModel.transform(testData)

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluatorgbt = MulticlassClassificationEvaluator(
    labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracygbt = evaluatorgbt.evaluate(predictionsgbt)
print("Test Error = %g" % (1.0 - accuracygbt))
print("Precisiòn = %g" % (accuracygbt*100))
evaluatorgbt.evaluate(predictionsgbt)

In [ ]:
from pyspark.ml.classification import LogisticRegression
lr = LogisticRegression(featuresCol = 'features', labelCol = 'label', maxIter=30)
lrModel = lr.fit(trainingData)
predictionslr = lrModel.transform(testData)

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluatorlr = MulticlassClassificationEvaluator(
    labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracylr = evaluatorlr.evaluate(predictionslr)
print("Test Error = %g" % (1.0 - accuracylr))
print("Precisiòn = %g" % (accuracylr*100))
evaluatorlr.evaluate(predictionslr)

In [ ]:
sc.stop()

# Referencias

1. Documentación de scikit-learn. http://scikit-learn.org/stable/index.html
2. Precision y Recall. https://en.wikipedia.org/wiki/Precision_and_recall
3. Machine Learning 101 (Google). https://docs.google.com/presentation/d/1kSuQyW5DTnkVaZEjGYCkfOxvzCqGEFzWBy4e9Uedd9k/preview?imm_mid=0f9b7e&cmp=em-data-na-na-newsltr_20171213#slide=id.g168a3288f7_0_58
4. WEKA (un programa visual con clasificadores y otras herramientas para ML). https://www.cs.waikato.ac.nz/ml/weka/
5. Curso de Data Mining con WEKA. https://www.cs.waikato.ac.nz/ml/weka/mooc/dataminingwithweka/
6. Una buena ayuda para esta prueba es: https://towardsdatascience.com/machine-learning-with-pyspark-and-mllib-solving-a-binary-classification-problem-96396065d2aa